In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import pandas as pd
import corner

# For GECCO formatting
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True

# Different length of backoff

In [ ]:
dirname = '../../ESR/esr/function_library/core_maths/'
all_p = [[], [], []]

for c in range(1, 11):
    
    print(c)

    logpar = np.loadtxt(dirname + f'compl_{c}/logconst_{c}.txt')
    
    for i in range(len(all_p)):
        p = np.loadtxt(dirname + f'compl_{c}/katz_codelen_{i+1}_{c}.txt')
        p -= logpar
        all_p[i] += list(p)
print('Loaded')
      
# Get finite values only
all_p = np.array(all_p, dtype=float)
m = np.ones(all_p.shape[1], dtype=bool) 
for i in range(all_p.shape[0]):
    m *= np.isfinite(all_p[i])
all_p = all_p[:,m]
print('Got finite')
        

In [ ]:
labs = [f'$- \log P (f_i | n={i+1})$' for i in range(all_p.shape[0])]
    
fig, axs = plt.subplots(all_p.shape[0], all_p.shape[0], figsize=(6,6), sharex=True)

for i in range(all_p.shape[0]):
    for j in range(i+1, all_p.shape[0]):
        plt.setp(axs[i,j].get_xticklines(), visible=False)

print('Making corner')
corner.corner(all_p.transpose(),
            labels=labs,
            fig=fig,
            label_kwargs={'fontsize':12},
            plot_contours=False,
            plot_density=False,
            plot_datapoints=False)

bins = np.logspace(0, np.log10(all_p.shape[1]))

print('Hexbin')
for i in range(all_p.shape[0]-1):
    for j in range(i+1, all_p.shape[0]):
        print(i, j)
        hb = axs[j,i].hexbin(all_p[i,:], all_p[j,:], mincnt=1, gridsize=40, bins='log')
        x = axs[j,i].get_xlim()
        axs[j,i].plot(x, x, color='r', lw=1)
        
for i in range(all_p.shape[0]):
    axs[i,i].clear()
    for j in range(all_p.shape[0]):
        if i == j:
            kwargs = {'color':'k',
                    'zorder':1,
                    'alpha':1,
                    'lw':1,
            }
        else:
            kwargs = {'color':'r',
                    'zorder':0,
                    'alpha':0.5,
                    'lw':0.75,
            }
        axs[i,i].hist(all_p[j,:], bins=30, density=True, histtype='step', **kwargs)
    axs[i,i].set_yticks([])
    
for i in range(axs.shape[0]):
    axs[-1,i].set_xlabel(r'%s'%labs[i])

ax2_divider = make_axes_locatable(axs[-2,-1])
cax2 = ax2_divider.append_axes("left", size="7%", pad="2%")
cb = fig.colorbar(hb, cax=cax2, orientation="vertical")
cb.set_label(r'Number of functions')


axs[0,0].set_xlim(0, 50)

fig.savefig('../figs/backoff_length.pdf', bbox_inches='tight', transparent=True)

# Benchmark testing

In [ ]:
name = 'korns_4'; true_eq = 'a0 + a1*sin(x)'
all_N = np.array([10, 30, 100, 300, 1000, 3000, 10000])
frac_sigx = 0.5
nsamp = 5
method_labels = ['Likelihood', 'Score', 'MDL', 'MDL Language Model', 'MDL FBF + Language Model', 'Bayes FBF + Language Model']
all_linthresh = [0.1, 1e5, 1, 1, 1, 1]

delta_loss = np.empty((6, len(all_N), nsamp))
has_truth = np.empty(delta_loss.shape, dtype=bool)

for i, nx in enumerate(all_N):
    for samp in range(nsamp):
        fname = f'{name}_{nx}_{frac_sigx}_{samp}'
        print(fname)
        d = pd.read_csv(f'../scripts/output/output_{fname}/selection_summary.csv', delimiter=';')
        for j in range(delta_loss.shape[0]):
            f0 = d['f0'][j]
            f1 = d['f1'][j]
            ftrue = d['ftrue'][j]
            has_truth[j,i,samp] = ((f0 == ftrue) or (f1 == ftrue))
            if f0 == ftrue:
                delta_loss[j,i,samp] = d['loss1'][j] - d['loss0'][j]
            elif f1 == ftrue:
                delta_loss[j,i,samp] = d['loss0'][j] - d['loss1'][j]
            else:
                delta_loss[j,i,samp] = d['loss0'][j] - d['losstrue'][j]

fig, axs = plt.subplots(2, int(np.ceil(delta_loss.shape[0]/2)), figsize=(10,4), sharex=True)
for i in range(axs.shape[0]):
    axs[i,0].set_ylabel('Preference for the truth')
for i in range(axs.shape[1]):
    axs[-1,i].set_xlabel('Number of data points')
axs = axs.flatten()
cm = plt.get_cmap('Set1')
for i in range(delta_loss.shape[0]):
    for j in range(delta_loss.shape[2]):
        axs[i].semilogx(all_N[has_truth[i,:,j]], delta_loss[i,has_truth[i,:,j],j], '.', color=cm(0))
        axs[i].semilogx(all_N[~has_truth[i,:,j]], delta_loss[i,~has_truth[i,:,j],j], '.', color=cm(1))
    axs[i].set_yscale('symlog', linthresh=all_linthresh[i])
    axs[i].axhline(y=0, color='k')
    axs[i].set_title(method_labels[i])
    axs[i].tick_params(labelbottom=True)
    

fig.tight_layout()

ntop2 = np.sum(has_truth, axis=(1,2))
for i in range(len(ntop2)):
    print('Method %s: %i of %i have truth in top 2'%(method_labels[i],ntop2[i],nsamp*len(all_N)))
        

In [ ]:
all_name = ['nguyen_8', 'korns_4', 'korns_7']
all_truth = {
    'nguyen_8':r'$y = \sqrt{x}$',
    'korns_4':r'$y = -2.13 + 0.13 \sin(x)$',
    'korns_7':r'$y = 213.81 (1 - e^{-0.547 x})$',
}

names_as_columns = False; figsize=(10,4); hspace=0.15; wspace=0.4
names_as_columns = True; figsize = (10,5); hspace=0.12; wspace=0.2

all_N = np.array([10, 30, 100, 300, 1000, 3000, 10000])
xticks = [10, 100, 1000, 10000]
frac_sigx = 0.5
nsamp = 5
method_labels = ['Likelihood', 'Score', 'MDL', 'MDL\n+LM', 
                 'MDL\n+FBF\n+LM', 'Bayes\n+FBF\n+LM']

if names_as_columns:
    fig, axs = plt.subplots(len(method_labels), len(all_name), figsize=figsize, sharex=True)
else:
    fig, axs = plt.subplots(len(all_name), len(method_labels), figsize=figsize, sharex=True)
cm = plt.get_cmap('Set1')

for bnum, name in enumerate(all_name):
    
    print(f'\n{name}')

    if names_as_columns:
        axs[0, bnum].set_title(r'%s: '%name + all_truth[name])
    else:
        axs[bnum, 0].set_ylabel(name)
    
    delta_loss = np.empty((len(method_labels), len(all_N), nsamp))
    has_truth = np.empty(delta_loss.shape, dtype=bool)
    truth_top = np.empty(delta_loss.shape, dtype=bool)

    for i, nx in enumerate(all_N):
        for samp in range(nsamp):
            fname = f'{name}_{nx}_{frac_sigx}_{samp}'
            d = pd.read_csv(f'../scripts/output/output_{fname}/selection_summary.csv', delimiter=';')
            for j in range(delta_loss.shape[0]):
                f0 = d['f0'][j]
                f1 = d['f1'][j]
                ftrue = d['ftrue'][j]
                has_truth[j,i,samp] = ((f0 == ftrue) or (f1 == ftrue))
                truth_top[j,i,samp] = (f0 == ftrue)
                if j == 1:
                    if f0 == ftrue:
                        delta_loss[j,i,samp] = np.log(np.abs(d['loss1'][j])) - np.log(np.abs(d['loss0'][j]))
                    elif f1 == ftrue:
                        delta_loss[j,i,samp] = np.log(np.abs(d['loss0'][j])) - np.log(np.abs(d['loss1'][j]))
                    else:
                        delta_loss[j,i,samp] = np.log(np.abs(d['loss0'][j])) - np.log(np.abs(d['losstrue'][j]))
                    delta_loss[j,i,samp] *= -1
                else:
                    if f0 == ftrue:
                        delta_loss[j,i,samp] = d['loss1'][j] - d['loss0'][j]
                    elif f1 == ftrue:
                        delta_loss[j,i,samp] = d['loss0'][j] - d['loss1'][j]
                    else:
                        delta_loss[j,i,samp] = d['loss0'][j] - d['losstrue'][j]
                    if not np.isfinite(delta_loss[j,i,samp]):
                        print("BAD", j, name, nx, samp)
    
    for i in range(delta_loss.shape[0]):
        for j in range(delta_loss.shape[2]):
            if names_as_columns:
                axs[i,bnum].semilogx(all_N[has_truth[i,:,j]], delta_loss[i,has_truth[i,:,j],j], '.', color=cm(0))
                axs[i,bnum].semilogx(all_N[~has_truth[i,:,j]], delta_loss[i,~has_truth[i,:,j],j], '.', color=cm(1))
            else:
                axs[bnum, i].semilogx(all_N[has_truth[i,:,j]], delta_loss[i,has_truth[i,:,j],j], '.', color=cm(0))
                axs[bnum, i].semilogx(all_N[~has_truth[i,:,j]], delta_loss[i,~has_truth[i,:,j],j], '.', color=cm(1))
        if names_as_columns:
            axs[i,bnum].axhline(y=0, color='k')
            axs[i,bnum].set_xticks(xticks)
        else:
            axs[bnum,i].axhline(y=0, color='k')
            axs[bnum,i].set_xticks(xticks)
        
    ntop2 = np.sum(has_truth, axis=(1,2))
    truth_top = np.sum(truth_top, axis=(1,2))
    for i in range(len(ntop2)):
        print('Method %s: %i of %i have truth in top 2'%(method_labels[i].replace('\n',''),ntop2[i],nsamp*len(all_N)))
    print('')    
    for i in range(len(ntop2)):
        print('Method %s: %i of %i has truth top'%(method_labels[i].replace('\n',''),truth_top[i],nsamp*len(all_N)))
        
for ax in axs.flatten():
    ymin, ymax = ax.get_ylim()
    if ymax < 0:
        ymax = abs(ymin)
    else:
        ymax = max(abs(ymin), abs(ymax))
    ymax *= 1.1
    ax.set_ylim(-ymax, ymax)

if names_as_columns:
    for i in range(axs.shape[0]):
        axs[i,0].set_ylabel(method_labels[i], fontsize=10)
else:
    for i in range(axs.shape[1]):
        axs[0,i].set_title(method_labels[i], fontsize=10)
        
for i in range(axs.shape[1]):
    axs[-1,i].set_xlabel('Number of data points')
fig.tight_layout()
fig.subplots_adjust(hspace=hspace, wspace=wspace)
fig.align_ylabels(axs[:,0])
        
fig.savefig('../figs/benchmark_results.pdf', bbox_inches='tight', transparent=True)